<a href="https://colab.research.google.com/github/trie0000/external/blob/main/saikyo_code_20260104_1938.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""DL_Basic_2025_Competition_NYUv2_baseline.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/17t7uAU0aST5aUt6sIJCqFneGlQrxFrJY

# Deep Learning 基礎講座　最終課題: NYUv2 セマンティックセグメンテーション

## 概要
RGB画像から、画像内の各ピクセルがどのクラスに属するかを予測するセマンティックセグメンテーションタスク.

### データセット
- データセット: NYUv2 dataset
- 訓練データ: 795枚
- テストデータ: 654枚
- 入力: RGB画像 + 深度マップ（元画像サイズは可変）
- 出力: 13クラスのセグメンテーションマップ
- 評価指標: Mean IoU (Intersection over Union)

### データセットの詳細（[NYU Depth Dataset V2](https://cs.nyu.edu/~fergus/datasets/nyu_depth_v2.html)）
- 画像は屋内シーンを撮影したもので、家具や壁、床などの物体が含まれています.
- 各画像に対して13クラスのセグメンテーションラベルが提供されます.
- データは以下のディレクトリ構造で提供:
```
data/NYUv2/
├─train/
│  ├─image/      # RGB画像
│  │    000000.png
│  │    ...
│  │
│  ├─depth/      # 深度マップ
│  │    000000.png
│  │    ...
│  │
│  └─label/      # 13クラスセグメンテーション（教師ラベル）
│       000000.png
│       ...
└─test/
   ├─image/      # RGB画像
   │    000000.png
   │    ...
   │  ├─depth/   # 深度マップ
   │    000000.png
   │    ...
```

### タスクの詳細
- 入力のRGB画像と深度マップから、各ピクセルが13クラスのどれに属するかを予測するタスクです.
- 評価はMean IoUを使用します．
  - 各クラスごとにIoUを計算し、その平均を取ります.
  - IoUは以下の式で計算:
  $$IoU = \frac{TP}{TP + FP + FN}$$
    - TP: True Positive（正しく予測されたピクセル数）
    - FP: False Positive（誤って予測されたピクセル数）
    - FN: False Negative（見逃したピクセル数）

### 前処理
- 入力画像は512×512にリサイズされます.
- ピクセル値は0-1に正規化されます.
- セグメンテーションラベルは0-12の整数値（13クラス）です．
  - 255はignore index（評価から除外）

### 提出形式
- テスト画像（RGB + Depth）の各ピクセルに対してクラス（0~12）を予測したものをnumpy配列として保存されます.
- ファイル名: `submission.npy`
- 配列の形状: [テストデータ数, 高さ, 幅]
- 各ピクセルの値: 0-12の整数（予測クラス）

## 考えられる工夫の例
- 事前学習モデルの fine-tuning
    - ImageNetなどで事前学習されたモデルを本データセットでfine-tuningすることで性能向上が見込めます.
- 損失関数の再設計
    - クラスごとの出現頻度に応じて損失を補正するように損失関数を設計すると、クラス分布の不均衡に対してロバストな学習ができます.
- 画像の前処理
    - RandomResizedCrop / Flip / ColorJitter 等のデータ拡張を追加することで，汎化性能の向上が見込めます．

## 修了要件を満たす条件
- ベースラインでは，omnicampus 上での性能評価において， 38.2% となります．したがって，ベースラインである 38.2% を超えた提出のみ，修了要件として認めます．
- ベースラインから改善を加えることで， 50%以上に性能向上することを運営で確認しています．こちらを 1つの指標として取り組んでみてください．

## 注意点
- 最終的な予測モデルは，**配布している訓練データを用いて学習**（ファインチューニング含む）したものとしてください．
- 学習を行わず，**事前学習済みモデルの知識のみを利用した推論は禁止**します．
（例: ChatGPT 等の LLM に入力して推論を得るのみ）

### 事前学習モデルの利用
許可される事項
- **構成要素としての事前学習モデルの利用**: 自身で実装したアーキテクチャの一部（特徴抽出，埋め込みなど）として事前学習モデル（BERT，ViT など）を利用することは可能です．
- **ファインチューニング**: 上記の用途で利用している事前学習モデルのファインチューニングは可能です．

禁止される事項
- **タスク解決用の事前学習モデルの利用**: transformers などで提供されている，対象タスクを直接解くための事前学習モデルでそのまま推論のみ，またはファインチューニングのみで利用することは禁止とします．
  - 禁止事項の例: VQA タスクを直接解くための事前学習モデルを VQA タスクで利用する．

### データの準備
データをダウンロードした際に，google drive したため，利用するために google drive をマウントする必要があります．また， drive 上で展開することができないため，/content ディレクトリ下にコピーし "data.zip" を展開します．
google drive 上に "data.zip" が配置されていない場合は実行できません．google drive 上に "data.zip" (**831MB**) を配置することが可能であれば，"data_download.ipynb" を先に実行してください．難しい場合は，omnicampus 演習環境を利用してください．．
"""

# omnicampus 上では 4 セル目まで実行不要
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

# データダウンロード用の notebook にてgoogle drive への保存後，
# 反映に時間がかかる可能性がありますので，google drive のマウント後，
# data.zip がディレクトリ内にあることを確認してから実行してください．
# data.zip を /content 下にコピーする
!cp "/content/drive/MyDrive/data.zip" "/content"

# Commented out IPython magic to ensure Python compatibility.
# カレントディレクトリ下のファイル群を確認
# data.zip が表示されれば問題ないです
# %ls

# データを解凍する
!unzip data.zip
!mkdir data
!mv train test data/

"""omnicampus 演習環境では，data_download.ipynb のマウント，zip 化，drive へのコピーを実行しないことで，"data.zip" を解凍した形で配置されます．したがって，data ディレクトリが存在するディレクトリをカレントディレクトリとするだけで良いです．


"""

# Commented out IPython magic to ensure Python compatibility.
# omnicampus 実行用
# 以下の例では/workspace/Segmentation/split_data_scripts/omnicampus に data ディレクトリがあると想定
# %cd /workspace/Segmentation/split_data_scripts_omnicampus

# omnicampus 実行用
!pip install h5py scikit-image

"""# import library"""

Mounted at /content/drive
Archive:  data.zip
  inflating: data/train/image/000600.png  
  inflating: data/train/image/000320.png  
  inflating: data/train/image/000491.png  
  inflating: data/train/image/000502.png  
  inflating: data/train/image/000129.png  
  inflating: data/train/image/000044.png  
  inflating: data/train/image/000652.png  
  inflating: data/train/image/000919.png  
  inflating: data/train/image/000528.png  
  inflating: data/train/image/000853.png  
  inflating: data/train/image/000177.png  
  inflating: data/train/image/000584.png  
  inflating: data/train/image/001319.png  
  inflating: data/train/image/000597.png  
  inflating: data/train/image/000223.png  
  inflating: data/train/image/001350.png  
  inflating: data/train/image/000404.png  
  inflating: data/train/image/000488.png  
  inflating: data/train/image/000268.png  
  inflating: data/train/image/000481.png  
  inflating: data/train/image/000341.png  
  inflating: data/train/image/000159.png  
  inflati

'# import library'

In [15]:
%ls -l /content/data
%pip install transformers albumentations timm
%pip install -U albumentations

total 8
drwxr-xr-x 4 root root 4096 Jan  4 03:11 test/
drwxr-xr-x 5 root root 4096 Jan  4 03:11 train/


In [30]:
# -*- coding: utf-8 -*-
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import time
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
from zipfile import ZipFile, ZIP_DEFLATED

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision import models
from torch.amp import autocast, GradScaler

import albumentations as A

# =========================
# Settings & Seed
# =========================
CLASS_NAMES = ["Bed", "Book", "Ceiling", "Chair", "Floor", "Cabinet", "Object", "Picture", "Sofa", "Desk", "TV", "Wall", "Window"]

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

# =========================
# Helper Functions
# =========================
def estimate_height_from_depth(depth_np):
    H, W = depth_np.shape
    y_grid = np.linspace(0, 1, H).reshape(H, 1).repeat(W, axis=1).astype(np.float32)
    height_map = y_grid * depth_np
    max_val = height_map.max()
    if max_val > 0:
        height_map /= max_val
    return height_map.astype(np.float32)

# =========================
# Dataset Class (Same as original)
# =========================
class NYUv2Dataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None, return_label=True):
        self.split = split
        self.transform = transform
        self.return_label = return_label
        src_split = 'train' if split in ['train', 'val'] else 'test'
        self.images_dir = os.path.join(root_dir, src_split, 'image')
        self.depths_dir = os.path.join(root_dir, src_split, 'depth')
        self.labels_dir = os.path.join(root_dir, src_split, 'label') if src_split == 'train' else None
        self.filenames = sorted([f for f in os.listdir(self.images_dir) if f.endswith('.png')])

        self.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
        self.std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
        self.d_mean, self.d_std = 0.5, 0.25

    def __len__(self): return len(self.filenames)

    def __getitem__(self, idx):
        fname = self.filenames[idx]
        rgb = np.array(Image.open(os.path.join(self.images_dir, fname)).convert('RGB'))
        depth = np.array(Image.open(os.path.join(self.depths_dir, fname)))
        if len(depth.shape) == 3: depth = depth[:, :, 0]

        depth = depth.astype(np.float32) / (65535.0 if depth.max() > 255 else 255.0)
        h_map = estimate_height_from_depth(depth)

        label = np.array(Image.open(os.path.join(self.labels_dir, fname))) if self.labels_dir and self.return_label else np.zeros(depth.shape, dtype=np.int32)

        if self.transform:
            augmented = self.transform(image=rgb, depth=depth, height=h_map, mask=label)
            rgb, depth, h_map, label = augmented['image'], augmented['depth'], augmented['height'], augmented['mask']

        rgb = (rgb.astype(np.float32) / 255.0 - self.mean) / self.std
        depth = (depth - self.d_mean) / self.d_std
        h_map = (h_map - self.d_mean) / self.d_std

        rgb_t = torch.from_numpy(rgb.transpose(2, 0, 1)).float()
        depth_t = torch.from_numpy(depth).unsqueeze(0).float()
        h_t = torch.from_numpy(h_map).unsqueeze(0).float()

        input_tensor = torch.cat([rgb_t, depth_t, h_t], dim=0)
        return (input_tensor, fname) if self.split == 'test' else (input_tensor, torch.from_numpy(label).long())

# =========================
# Losses (Same as original)
# =========================
class DiceLoss(nn.Module):
    def __init__(self, n_classes=13, smooth=1e-5, ignore_index=255):
        super().__init__()
        self.n_classes = n_classes
        self.smooth = smooth
        self.ignore_index = ignore_index

    def forward(self, pred, target):
        pred = F.softmax(pred, dim=1)
        mask = (target != self.ignore_index)
        target_valid = target.clone()
        target_valid[~mask] = 0
        target_one_hot = F.one_hot(target_valid, self.n_classes).permute(0, 3, 1, 2).float()

        m_expand = mask.unsqueeze(1).expand_as(pred)
        pred, target_one_hot = pred * m_expand, target_one_hot * m_expand

        intersection = (pred * target_one_hot).sum(dim=(2, 3))
        union = pred.sum(dim=(2, 3)) + target_one_hot.sum(dim=(2, 3))
        dice = (2. * intersection + self.smooth) / (union + self.smooth)
        return 1 - dice.mean()

# =========================
# Model Definition (Same as original)
# =========================
class ConvBNReLU(nn.Module):
    def __init__(self, in_ch, out_ch, k=3, p=1, d=1):
        super().__init__()
        self.block = nn.Sequential(nn.Conv2d(in_ch, out_ch, k, padding=p, dilation=d, bias=False), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True))
    def forward(self, x): return self.block(x)

class ASPP(nn.Module):
    def __init__(self, in_ch, out_ch=256):
        super().__init__()
        self.b1 = ConvBNReLU(in_ch, out_ch, 1, 0)
        self.b2 = ConvBNReLU(in_ch, out_ch, 3, 6, 6)
        self.b3 = ConvBNReLU(in_ch, out_ch, 3, 12, 12)
        self.b4 = ConvBNReLU(in_ch, out_ch, 3, 18, 18)
        self.b5 = nn.Sequential(nn.AdaptiveAvgPool2d(1), ConvBNReLU(in_ch, out_ch, 1, 0))
        self.proj = nn.Sequential(ConvBNReLU(out_ch * 5, out_ch, 1, 0), nn.Dropout(0.1))
    def forward(self, x):
        h, w = x.shape[2:]
        feat1, feat2, feat3, feat4 = self.b1(x), self.b2(x), self.b3(x), self.b4(x)
        feat5 = F.interpolate(self.b5(x), size=(h, w), mode='bilinear', align_corners=False)
        return self.proj(torch.cat([feat1, feat2, feat3, feat4, feat5], dim=1))

class ResNeXtDeepLabV3Plus(nn.Module):
    def __init__(self, num_classes=13, in_channels=5):
        super().__init__()
        backbone = models.resnext101_32x8d(weights=models.ResNeXt101_32X8D_Weights.IMAGENET1K_V1)

        old_conv = backbone.conv1
        new_conv = nn.Conv2d(in_channels, old_conv.out_channels, 7, stride=2, padding=3, bias=False)
        with torch.no_grad():
            new_conv.weight[:, :3] = old_conv.weight
            new_conv.weight[:, 3:] = old_conv.weight.mean(dim=1, keepdim=True)

        self.enc0 = nn.Sequential(new_conv, backbone.bn1, backbone.relu)
        self.pool = backbone.maxpool
        self.enc1, self.enc2, self.enc3, self.enc4 = backbone.layer1, backbone.layer2, backbone.layer3, backbone.layer4

        self.aspp = ASPP(2048, 256)
        self.low_proj = nn.Sequential(nn.Conv2d(256, 48, 1, bias=False), nn.BatchNorm2d(48), nn.ReLU(inplace=True))
        self.decoder = nn.Sequential(ConvBNReLU(304, 256, 3, 1), ConvBNReLU(256, 256, 3, 1), nn.Conv2d(256, num_classes, 1))

    def forward(self, x):
        h, w = x.shape[2:]
        x = self.enc0(x); x = self.pool(x)
        low_level = self.enc1(x)
        x = self.enc4(self.enc3(self.enc2(low_level)))
        x = F.interpolate(self.aspp(x), size=low_level.shape[2:], mode='bilinear', align_corners=False)
        x = self.decoder(torch.cat([x, self.low_proj(low_level)], dim=1))
        return F.interpolate(x, size=(h, w), mode='bilinear', align_corners=False)

# =========================
# Main Training Loop
# =========================
def main():
    dataset_root = "/content/data"
    img_size = 768
    batch_size = 16
    epochs = 50
    lr = 1e-4
    set_seed(42)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    timestamp = time.strftime("%Y%m%d%H%M%S")
    best_model_path = f"best_model_v2_{timestamp}.pt"

    # Transforms (Same as original)
    train_aug = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.8, rotate_limit=15, p=0.7, border_mode=0),
        A.ColorJitter(p=0.5),
        A.Resize(img_size, img_size)
    ], additional_targets={'depth': 'image', 'height': 'image'})

    val_aug = A.Compose([A.Resize(img_size, img_size)], additional_targets={'depth': 'image', 'height': 'image'})

    # =====================
    # DataLoaders (★ BUG FIX: Proper train/val split)
    # =====================
    full_dataset = NYUv2Dataset(dataset_root, split='train', transform=None)
    n_total = len(full_dataset)
    n_val = int(n_total * 0.1)
    n_train = n_total - n_val

    # Reproducible split
    indices = list(range(n_total))
    random.shuffle(indices)
    train_indices = indices[:n_train]
    val_indices = indices[n_train:]

    # Create separate dataset instances with different transforms
    train_dataset = NYUv2Dataset(dataset_root, split='train', transform=train_aug)
    val_dataset = NYUv2Dataset(dataset_root, split='train', transform=val_aug)

    train_loader = DataLoader(
        Subset(train_dataset, train_indices),
        batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True, drop_last=True
    )
    # ★ FIX: Use val_indices instead of entire dataset
    val_loader = DataLoader(
        Subset(val_dataset, val_indices),
        batch_size=8, shuffle=False, num_workers=4, pin_memory=True
    )
    test_loader = DataLoader(
        NYUv2Dataset(dataset_root, split='test', transform=val_aug, return_label=False),
        batch_size=1
    )

    print(f"Train: {len(train_indices)}, Val: {len(val_indices)}, Test: {len(test_loader.dataset)}")

    # Weights (Same as original)
    weights = torch.tensor([1.0, 5.5, 0.6, 2.2, 0.6, 1.0, 1.8, 1.5, 1.5, 3.0, 3.0, 0.5, 1.0]).to(device)

    model = ResNeXtDeepLabV3Plus().to(device)
    criterion_ce = nn.CrossEntropyLoss(weight=weights, ignore_index=255)
    criterion_dice = DiceLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-3)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    scaler = GradScaler("cuda")

    best_miou = 0.0
    for epoch in range(epochs):
        model.train()
        for x, label in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            x, label = x.to(device), label.to(device)
            optimizer.zero_grad(set_to_none=True)
            with autocast("cuda"):
                out = model(x)
                loss = 0.4 * criterion_ce(out, label) + 0.6 * criterion_dice(out, label)
            scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        scheduler.step()

        # Validation
        model.eval(); cm = torch.zeros((13, 13), dtype=torch.long)
        with torch.no_grad():
            for x, label in val_loader:
                pred = model(x.to(device)).argmax(1).cpu().view(-1)
                target = label.view(-1)
                mask = (target != 255)
                cm += torch.bincount(target[mask] * 13 + pred[mask], minlength=169).view(13, 13)

        tp = torch.diag(cm).float(); iou = tp / (cm.sum(0) + cm.sum(1) - tp + 1e-8)
        miou = iou.mean().item()
        print(f"Epoch {epoch+1}: mIoU={miou:.4f}")
        print(f"  Per-class: {', '.join([f'{CLASS_NAMES[i]}:{iou[i]:.3f}' for i in range(13)])}")

        if miou > best_miou:
            best_miou = miou
            torch.save(model.state_dict(), best_model_path)
            print(f"  -> New best: {best_miou:.4f}")

    # Submission
    print("Generating submission...")
    model.load_state_dict(torch.load(best_model_path))
    model.eval(); preds = []
    with torch.no_grad():
        for x, _ in tqdm(test_loader):
            acc = torch.zeros((1, 13, 512, 512), device=device)
            for s in [1.0, 1.25]:
                h_s, w_s = int(img_size*s), int(img_size*s)
                x_s = F.interpolate(x.to(device), size=(h_s, w_s), mode='bilinear')
                out = model(x_s) + torch.flip(model(torch.flip(x_s, [3])), [3])
                acc += F.interpolate(out, size=(512, 512), mode='bilinear')
            preds.append(acc.argmax(1).cpu().numpy().astype(np.uint8))

    submission = np.concatenate(preds, axis=0)
    np.save("submission.npy", submission)
    with ZipFile("submission.zip", "w", ZIP_DEFLATED) as zf: zf.write("submission.npy")
    print(f"Final shape: {submission.shape}, Best mIoU: {best_miou:.4f}")

if __name__ == "__main__": main()

Train: 716, Val: 79, Test: 654


Epoch 1: 100%|██████████| 44/44 [00:21<00:00,  2.03it/s]


Epoch 1: mIoU=0.3188
  Per-class: Bed:0.511, Book:0.094, Ceiling:0.000, Chair:0.355, Floor:0.767, Cabinet:0.291, Object:0.363, Picture:0.376, Sofa:0.072, Desk:0.239, TV:0.080, Wall:0.635, Window:0.361
  -> New best: 0.3188


Epoch 2: 100%|██████████| 44/44 [00:20<00:00,  2.11it/s]


Epoch 2: mIoU=0.3753
  Per-class: Bed:0.520, Book:0.117, Ceiling:0.000, Chair:0.373, Floor:0.820, Cabinet:0.315, Object:0.394, Picture:0.402, Sofa:0.244, Desk:0.284, TV:0.275, Wall:0.674, Window:0.462
  -> New best: 0.3753


...

Epoch 49: mIoU=0.6677
  Per-class: Bed:0.808, Book:0.184, Ceiling:0.679, Chair:0.681, Floor:0.929, Cabinet:0.625, Object:0.591, Picture:0.708, Sofa:0.670, Desk:0.426, TV:0.746, Wall:0.868, Window:0.765


Epoch 50: 100%|██████████| 44/44 [00:21<00:00,  2.08it/s]


Epoch 50: mIoU=0.6669
  Per-class: Bed:0.817, Book:0.181, Ceiling:0.682, Chair:0.684, Floor:0.930, Cabinet:0.626, Object:0.591, Picture:0.711, Sofa:0.659, Desk:0.424, TV:0.736, Wall:0.870, Window:0.761
Generating submission...


100%|██████████| 654/654 [01:32<00:00,  7.09it/s]


Final shape: (654, 512, 512), Best mIoU: 0.6689


In [ ]:
# ------------------
#    Evaluation
# ------------------

model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# 予測結果の生成
predictions = []

with torch.no_grad():
    print("Generating predictions...")
    for image, depth in tqdm(test_data):
        image, depth = image.to(device), depth.to(device)
        x = torch.cat((image, depth), dim=1)
        output = model(x)            # [Batch, num_classes, H, W]
        pred = output.argmax(dim=1)  # [Batch, H, W]
        predictions.append(pred.cpu())
predictions = torch.cat(predictions, dim=0)

predictions = predictions.cpu().numpy()
np.save('submission.npy', predictions)
print("Predictions saved to submission.npy")

"""## 提出方法

以下の3点をzip化し，Omnicampusの「最終課題 (セグメンテーション)」から提出してください．

- `submission.npy`
- `model.pt`や`model_best.pt`など，テストに使用した重み（拡張子は`.pt`のみ）
- 本Colab Notebook
"""

from zipfile import ZipFile, ZIP_DEFLATED

notebook_path = "/content/drive/MyDrive/Colab Notebooks/DL_Basic_2025_Competition_NYUv2_baseline.ipynb"

with ZipFile("submission.zip",
             mode="w",
             compression=ZIP_DEFLATED,
             compresslevel=9) as zf:
    zf.write("submission.npy")
    zf.write(model_path)
    zf.write(notebook_path,
             arcname="DL_Basic_2025_Competition_NYUv2_baseline.ipynb")